In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/redditresampled/rdResampled.csv


In [35]:
import pandas as pd 
df=pd.read_csv('/kaggle/input/redditresampled/rdResampled.csv')

In [36]:

df=df.drop(['Unnamed: 0'],axis=1)
df.head()

,title_post,Subreddit
0,The end of the road. I've never posted on redd...,depression
1,I can’t write comments. I think I should not c...,depression
2,Attempted hanging myself two years ago. So nea...,depression
3,"25/ f started deliberate self injury. Hi all, ...",depression
4,Venting I guess. I've been crying in my high s...,depression


In [37]:
df['Subreddit'].value_counts()

depression       35000
Anxiety          35000
bipolar          35000
BPD              35000
schizophrenia    35000
autism           35000
Name: Subreddit, dtype: int64

In [44]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [45]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification

In [94]:
X = df['title_post']
y = df['Subreddit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [53]:
print(len(X_train),len(X_test))

168000 42000


In [80]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [81]:
ls bert-base-uncased

ls: cannot access 'bert-base-uncased': No such file or directory


In [ ]:
train_encoded = tokenizer.batch_encode_plus(X_train.tolist(), truncation=True, padding=True, max_length=256, return_tensors='tf')
test_encoded = tokenizer.batch_encode_plus(X_test.tolist(), truncation=True, padding=True, max_length=256, return_tensors='tf')


In [87]:
num_examples = train_encoded['input_ids'].shape[0]
print("Number of examples:", num_examples)

Number of examples: 168000


In [86]:
example_input_ids = train_encoded['input_ids'][0]
print(example_input_ids)

tf.Tensor(
[  101  2064  1521  1056  3637  1012  4397  5117  1998 13847  2041  1012
  1012  1045  1521  1049  5117  2000  1996  2190  2711  2412  1010  1998
  2057  1521  2128  2275  2000  2131  2496  2279  2095   999  2002  1521
  1055  6919  1012  2002  1521  1055  2785  1010  1037  2524  7309  1010
  8884  1010  1037  2204  3244  2000  5390  2006  1010  2673  1045  1521
  2310  2412  2734  1999  1037  4256  1012  2061  2619  2425  2033  2339
  2572  1045 13847  2041  1998  3402  7995  2006  1523  4997  1524 12332
  1029  2093  2706  1999  2000  2256  3276  1010  2002  9962 23356 26207
  5225  2007  1037  5404  1999  1996  2482  1012  1045  1521  1049  1037
  2775  1997  2019 14813  1998  1045  7210  2012  2032  1999  4963  1998
 12721  1012  2002  1521  1055  1037  2406  2879  2007  2053  2610  1999
  2010  2212  1010  1045  1521  1049  1037  2103  2611  1012  2002  6719
  2106  2025  5382  2054  2002  2001  2725  3432  2138  2057  2020  2183
  2000  1996 13025  3573  1012  2002 178

In [95]:
label_mapping = {label: i for i, label in enumerate(df['Subreddit'].unique())}
y_train = y_train.map(label_mapping)
y_test = y_test.map(label_mapping)

In [117]:
label_mapping

{'depression': 0,
 'Anxiety': 1,
 'bipolar': 2,
 'BPD': 3,
 'schizophrenia': 4,
 'autism': 5}

In [100]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_mapping))

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [103]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


In [104]:
model.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_151 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  4614      
                                                                 
Total params: 109,486,854
Trainable params: 109,486,854
Non-trainable params: 0
_________________________________________________________________


In [105]:
import numpy as np

# Convert train_encoded and test_encoded to numpy arrays
train_inputs = train_encoded['input_ids'].numpy()
train_attention_mask = train_encoded['attention_mask'].numpy()
test_inputs = test_encoded['input_ids'].numpy()
test_attention_mask = test_encoded['attention_mask'].numpy()

# Convert y_train to numpy array
y_train = np.array(y_train)



In [110]:
batch_size = 16
history = model.fit(
    x=[train_inputs, train_attention_mask],
    y=y_train,
    batch_size=batch_size,
    epochs=3,
    validation_data=([test_inputs, test_attention_mask], y_test)
)

Epoch 1/3
10500/10500 [==============================] - 5467s 517ms/step - loss: 0.6040 - accuracy: 0.7913 - val_loss: 0.4940 - val_accuracy: 0.8284
Epoch 2/3
10500/10500 [==============================] - 5463s 520ms/step - loss: 0.4198 - accuracy: 0.8557 - val_loss: 0.4407 - val_accuracy: 0.8485
Epoch 3/3
10500/10500 [==============================] - 5426s 517ms/step - loss: 0.3138 - accuracy: 0.8923 - val_loss: 0.4516 - val_accuracy: 0.8489


In [112]:
# Saving model method 1

model.save('/kaggle/working/BERTmodel', save_format='tf')
#model loading
# import tensorflow as tf

# loaded_model = tf.keras.models.load_model('model')


In [113]:
# Saving model method 2
model.save_weights('/kaggle/working/BERTmodel_weights.h5')
#model loading
# import tensorflow as tf

# # Define the model architecture
# model = YourModelArchitecture()  # Replace with your model architecture

# # Load the saved weights
# model.load_weights('model_weights.h5')


In [115]:
input_ids = test_encoded['input_ids']
attention_mask = test_encoded['attention_mask']
token_type_ids = test_encoded['token_type_ids']

predictions = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask, 'token_type_ids': token_type_ids})



1313/1313 [==============================] - 423s 319ms/step


In [118]:
label_mapping_reverse = {0: 'depression', 1: 'Anxiety', 2: 'bipolar', 3: 'BPD', 4: 'schizophrenia', 5: 'autism'}

predicted_labels = tf.argmax(predictions.logits, axis=1)
predicted_class_names = [label_mapping_reverse[label.numpy()] for label in predicted_labels]


In [139]:
predicted_class_names[:5]

['autism', 'schizophrenia', 'Anxiety', 'Anxiety', 'schizophrenia']

In [142]:
y_test[:5]

[5, 4, 1, 1, 4]

In [137]:
print(type(predicted_class_names))
# y_test = y_test.tolist()
# type(y_test)

<class 'list'>


In [127]:
type(predicted_class_names)

list

In [144]:


# Define the mapping between numerical labels and class names
label_mapping = {0: 'depression', 1: 'Anxiety', 2: 'bipolar', 3: 'BPD', 4: 'schizophrenia', 5: 'autism'}

# Convert the numerical labels in y_test to class names
y_test_strings = [label_mapping[label] for label in y_test]



In [146]:
y_test_strings[:5]

['autism', 'schizophrenia', 'Anxiety', 'Anxiety', 'schizophrenia']

In [147]:
from sklearn.metrics import classification_report

# Calculate classification report
classification_metrics = classification_report(y_test_strings, predicted_class_names)
print(classification_metrics)


               precision    recall  f1-score   support

      Anxiety       0.83      0.86      0.84      6883
          BPD       0.74      0.83      0.78      6926
       autism       0.98      0.99      0.99      7069
      bipolar       0.83      0.77      0.80      7108
   depression       0.80      0.74      0.77      6950
schizophrenia       0.92      0.90      0.91      7064

     accuracy                           0.85     42000
    macro avg       0.85      0.85      0.85     42000
 weighted avg       0.85      0.85      0.85     42000

